## STEP 1: Ideation

In [ ]:
# Data pulling
import pandas as pd 
import numpy as np 
import yfinance as yf

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import (
    train_test_split,
    RandomizedSearchCV,
    GridSearchCV,
    TimeSeriesSplit,
    cross_val_score
)

# import classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Metrics
from sklearn.metrics import (
    precision_recall_curve,
    roc_curve,
    RocCurveDisplay,
    ConfusionMatrixDisplay
)
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    recall_score,
    precision_score,
    roc_auc_score,
    auc
)

from sklearn.metrics import (
    classification_report,
    confusion_matrix
)


## STEP 2: Data Collection

We will use the `yfinance` package to download daily trading data from Yahoo Finance. The recommended data should span a 5-year period, which is considered sufficient. The downloaded data will be saved in the `.csv` format and can be accessed later using the file name `SPY1D.csv`.

In [ ]:
# Download data for TSLA and store as csv file
spy = yf.download("SPY", start = '2008-10-16', end = '2023-10-16' , interval='1D')
spy.to_csv('SPY1D.csv')

In [ ]:
spy = pd.read_csv('../module_4/SPY1D.csv')

In [ ]:
# Verify the downloaded data
spy.info()

## STEP 3: EDA

Visualize asset path:

In [ ]:
import plotly.express as px

fig = px.line(spy, x= 'Date', y='Adj Close', labels = {'Adj Close': 'Close Price (USD)'}, title = 'S&P 500 ETF Trust (SPY) Daily')
fig.show();

### Calculate returns

We can plot the distribution of returns and the closing price movement to identify any trends or significant information regarding the returns that could be useful.

In [ ]:
spy['Returns'] = np.log(spy['Adj Close']).diff()

In [ ]:
from scipy.stats import norm

# Plot the return histogram
fig = plt.figure(figsize=(15, 7))
ax1 = fig.add_subplot(1, 1, 1)
spy['Returns'].hist(bins=50, ax=ax1)
ax1.set_xlabel('Return')
ax1.set_ylabel('Count')
ax1.set_title('Return distribution')

# Plot the normal distribution
mu = spy['Returns'].mean()
sigma = spy['Returns'].std()
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
plt.plot(x, norm.pdf(x, mu, sigma))
plt.show()

The return is definately not normally distributed. There is a high peak and very fat tails. 

### Feature Specify

Using the feature list table from the exam, we will generate features based on the historical data we have acquired. Additionally, I've included 10 lagged prices in the feature list, operating on the assumption that historical data may possess predictive capabilities.

In [ ]:
# Create features (predictors) list
features_list = []
# Intraday price range
spy['OC'] = spy['Open'] - spy['Close']
spy['HL'] = spy['High'] - spy['Low']
# Sign of return or momentum
spy['Sign'] = np.sign(spy.Returns)

# Append feature list
features_list.append('OC')
features_list.append('HL')
features_list.append('Sign')

# Pass Returns, Volatility
for r in range(10, 65, 5):
    spy['Ret_'+str(r)] = spy.Returns.rolling(r).sum()
    spy['Std_'+str(r)] = spy.Returns.rolling(r).std()
    features_list.append('Ret_'+str(r))
    features_list.append('Std_'+str(r))

# SMA and EMA
for a in range(20, 200, 10):
    spy['SMA_'+str(r)] = spy['Adj Close'].rolling(r).mean()
    spy['EMA_'+str(a)] = spy['Adj Close'].ewm(span = a).mean()
    features_list.append('SMA_'+str(r))
    features_list.append('EMA_'+str(r))

# Lag price
for lag in range(1, 10):
    spy['lag_' + str(lag)] = spy['Adj Close'].shift(lag)

# Drop NaN values
spy.dropna(inplace=True)

### Define target

In [ ]:
# Define Target
spy['Target'] = np.where(spy['Adj Close'].shift(-1) > 0.995 * spy['Adj Close'],1,0)
# Check output
spy.head(10)

I am going to split the data into the `train_set` and `test_set` and perform exploratory data analysis (EDA) and data cleaning exclusively on the `train_set` to prevent any potential data leakage from the EDA process.

In [ ]:
# Copy the original data
data = spy.copy().set_index('Date')

In [ ]:
# Specify the features matrix `X`
X = data.drop(['Open', 'Close', 'High', 'Low', 'Adj Close', 'Returns', 'Volume', 'Target'],axis=1)
X.info()

In [ ]:
# Define label or target vector `y`
y = data['Target']
y

In [ ]:
# Splitting the datasets into training and testing data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
# Output the train and test data size
print(f"Train and Test Size {len(X_train)}, {len(X_test)}")

### Imbalance class

Since this is a classification problem, it's important to check for any imbalances in our labels.

In [ ]:
# class frequency
c = y_train.value_counts()
c

The label is imbalanced. We will create a weight function and subsequently use it to address our problem when building a model.

In [ ]:
# class weight function
def cwts(label):
    c0, c1 = np.bincount(label)
    w0=(1/c0)*(len(label))/2 
    w1=(1/c1)*(len(label))/2 
    return {0: w0, 1: w1}

In [ ]:
# check class weights
class_weight = cwts(y_train)
class_weight

### Multi collinearity features

Collinear features can adversely affect our model's performance. We will create a function to help us identify and drop these features, and then apply it to our test dataset. Let's also visualize our correlation matrix using the `sns.heatmap()` method.

In [ ]:
plt.figure(figsize=(25, 22))

# Identify features that are highly correlated
sns.heatmap(X_train.corr()>0.9,
            annot=True,
            annot_kws={"size": 8},
            fmt=".2f",
            linewidth=.5,
            cmap="coolwarm",
            cbar=True); #cmap="crest", virids, magma

plt.title('Features Set Correlations');

Feature scaling is also a crucial factor in our model's accuracy. We need to scale the data before inputting it into our learning algorithm. We can easily identify features that require scaling by using the `sns.boxplot()` method.

In [ ]:
# study the distribution
fig, ax = plt.subplots(figsize=(14,8))
sns.boxplot(x='variable', y='value', data=pd.melt(X_train))
plt.xlabel(' ')
plt.title('Boxplot of Features');

Alternatively, we can identify features that require scaling by using the `pd.describe()` method.

In [ ]:
X_train.describe()

Some features exhibit significantly higher values compared to the others. For these features, we will use the `MinMaxScaler()` method to scale them appropriately.

## STEP 4: Cleaning Data

From our exploratory data analysis (EDA) process, we have identified multicollinear features. We will develop a function to eliminate these features and then implement it on our training data. Subsequently, we will apply the same function to our test data.

In [ ]:
# remove the first feature that is correlated with any other feature
def correlated_features(data, threshold=0.9):
    col_corr = set()
    corr_matrix = X_train.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [ ]:
# Get the list of remaining features
drop_correlated_features = correlated_features(X_train, threshold=0.9)

In [ ]:
# drop the highly correlated features
X_train_drop = X_train.drop(drop_correlated_features, axis=1)
X_train_drop.describe()

After removing most of the highly correlated features, it appears that past returns, past volatility, SMA (Simple Moving Average), OC (Open-Close), HL (High-Low), and Sign have significant predictive power.

In [ ]:
X_test_drop = X_test.drop(drop_correlated_features, axis=1)

## STEP 5: Transformation

We will visualize the scale of our data once more before proceeding with feature transformation.

In [ ]:
# study the distribution
plt.figure(figsize=(8, 4))
sns.boxplot(x='variable', y='value', data=pd.melt(X_train_drop))
plt.xlabel(' ')
plt.title('Boxplot of Features');

The only remaining feature with a high value is `SMA_60`. We will scale this feature using `MinMaxScaler()`.

In [ ]:
minmax = ColumnTransformer([
    ('scaled', MinMaxScaler(), ['SMA_60'])
],remainder = 'passthrough')

In [ ]:
# Fit and transform the data
sma_60 = minmax.fit_transform(X_train_drop)

In [ ]:
X_train_dropped_scaled = pd.DataFrame(
    sma_60, columns=minmax.get_feature_names_out(),
    index=X_train_drop.index)

In [ ]:
X_train_dropped_scaled.describe()

Let's visualize our scaled data once more.

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))
sns.boxplot(x='variable', y='value', data=pd.melt(X_train_dropped_scaled))
plt.xlabel('After scaled')
plt.title('Boxplot of Features');

It appears that the `OC` and `HL` columns contain a substantial number of outliers. We will employ the `RobustScaler` to transform these features.

In [ ]:
robust = ColumnTransformer([
        ('cols', RobustScaler(), ['OC','HL'])
    ],remainder = 'passthrough')

oc_hl = robust.fit_transform(X_train_drop)

X_train_dropped_scaled = pd.DataFrame(
    oc_hl, columns=robust.get_feature_names_out(),
    index=X_train_drop.index)

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))
sns.boxplot(x='variable', y='value', data=pd.melt(X_train_dropped_scaled.drop(['remainder__SMA_60'],axis=1)))
plt.xlabel('After scaled')
plt.title('Boxplot of Features');

Now we can construct a preprocessing transformer that applies the specified transformations to particular columns. We will fit and transform the training data and subsequently transform the test data.

In [ ]:
# Instantiate transformer
preprocessing = ColumnTransformer([
        ('MinMax', MinMaxScaler(), ['SMA_60']),
        ['Robust', RobustScaler(),['OC','HL'] ]
    ],remainder = 'passthrough')

In [ ]:
# Fit and transform train set
train_transformed = preprocessing.fit_transform(X_train_drop)
X_train_transformed = pd.DataFrame(
    train_transformed, columns=preprocessing.get_feature_names_out(),
    index=X_train_drop.index)

# Transform test set
test_transformed = preprocessing.transform(X_test_drop)
X_test_transformed = pd.DataFrame(
    test_transformed, columns=preprocessing.get_feature_names_out(),
    index=X_test_drop.index)

## STEP 6: Modeling

We will compare the default settings of some classifiers using the cross-validation technique to identify potential candidates for our final model. Additionally, we will use the `class_weight` parameter to address the previously identified class imbalance problem.

In [ ]:
# cross-validation
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
# specify estimators
random_state = 42
dtc = DecisionTreeClassifier(class_weight=class_weight)
rfc = RandomForestClassifier(max_depth = 5 ,class_weight=class_weight, random_state=random_state)
knn = KNeighborsClassifier()
gbc = GradientBoostingClassifier(random_state=random_state)
svc = SVC(class_weight=class_weight, random_state=random_state)


In [ ]:
# get cv scores
clf = [dtc, rfc, knn, gbc, svc]
for estimator in clf:
    score = cross_val_score(estimator, X_train_transformed, y_train, scoring = 'accuracy', cv=tscv, n_jobs=-1)
    print(f"The accuracy score of {estimator} is: {score.mean():0.4}")

It appears that the `RandomForestClassifier()` and `KNeighborsClassifier()` have the highest scores. Given that the `KNeighborsClassifier()` may not perform well with imbalanced classes, we will concentrate on building the model using the `RandomForestClassifier()`.

### Base Model

The default values for the parameters that determine the size of the trees (e.g., `max_depth`, `min_samples_leaf`, etc.) result in fully grown and unpruned trees, which have the potential to overfit our model. To address this, I will set `max_depth` to 5 and then fine-tune this hyperparameter later.

In [ ]:
base_model = RandomForestClassifier(max_depth = 5, class_weight=class_weight, random_state=random_state)
base_model.fit(X_train_transformed,y_train)
print (classification_report(y_train[-252:], base_model.predict(X_train_transformed[-252:])))

### Tuning Hyper-params

We will obtain all the parameters and define our hyperparameter grid.

In [ ]:
model = RandomForestClassifier(class_weight=class_weight, random_state=random_state, n_jobs=-1)

In [ ]:
model.get_params()

As mentioned earlier, we will include `max_depth`, `max_leaf_nodes`, and `n_estimators` in our hyperparameter grid for tuning to prevent overfitting. Additionally, since we are dealing with an imbalanced classification problem, we will experiment with different loss functions to determine their impact on model performance during the hyperparameter search.

In [ ]:
# Hyper parameter optimization
param_grid = {  'criterion': ['gini', 'entropy', 'log_loss'],
                'max_depth': [80, 90, 100, 110],
                'max_features': [2, 3],
                'min_samples_leaf': [3, 4, 5],
                'min_samples_split': [8, 10, 12],
                'n_estimators': [100, 200, 300, 1000]
            }

In [ ]:
# perform random search
gs = GridSearchCV(model, param_grid, scoring='f1', cv=tscv, verbose=0, n_jobs=-1)
gs.fit(X_train_transformed, y_train)

In [ ]:
# best parameters
gs.best_params_

In [ ]:
# best score
gs.best_score_

## STEP 7: Metrics

After fine-tuning our model and conducting a search for the best hyperparameters, we will evaluate our model's performance and compare it to our base model.

In [ ]:
# Refit the XGB Classifier with the best params
final_model = RandomForestClassifier(class_weight=class_weight, random_state=random_state, n_jobs=-1, **gs.best_params_)
final_model.fit(X_train_transformed, y_train)

In [ ]:
# Predicting the test dataset
y_pred = final_model.predict(X_test_transformed)
# Measure Accuracy
acc_train = accuracy_score(y_train, final_model.predict(X_train_transformed))
acc_test = accuracy_score(y_test, y_pred)
# Print Accuracy
print(f'\n Training Accuracy \t: {acc_train :0.4} \n Test Accuracy \t\t: {acc_test :0.4}')

Our final model outperforms the base model, but it appears to suffer from severe overfitting.

In [ ]:
# Cross validation score
score = cross_val_score(final_model,X_train_transformed,y_train,cv=tscv)
print(f'Mean CV Score : {score.mean():0.4}')

In [ ]:
# Plot feature importance
fig, ax = plt.subplots(figsize=(10,8))
feature_imp = pd.DataFrame({'Importance Score': final_model.feature_importances_,'Features': X_train_transformed.columns}).sort_values(by='Importance Score', ascending=False)
sns.barplot(x=feature_imp['Importance Score'], y=feature_imp['Features'])
ax.set_title('Features Importance');

It appears that the most important features are the asset returns, volatilities, and H-L (High-Low) values. There is some predictive power in the `SMA_60` feature, while the `Sign` feature contributes almost no predictive power.

In [ ]:
# Classification Report
print(classification_report(y_test, y_pred))

In [ ]:
# Display confussion matrix
disp = ConfusionMatrixDisplay.from_estimator(
        final_model,
        X_test_transformed,
        y_test,
        display_labels=final_model.classes_,
        cmap=plt.cm.Blues
    )   
disp.ax_.set_title('Final Model')
plt.show()

Our model is performing well when predicting the majority class but struggles when predicting the minority class.

In [ ]:
# Display ROCCurve
disp_roc = RocCurveDisplay.from_estimator(
        final_model,
        X_test_transformed,
        y_test,
        name='Tuned Random Forest')
disp_roc.ax_.set_title('ROC Curve')
plt.plot([0,1], [0,1], linestyle='--')
plt.show()

When analyzing the ROC curve, it becomes evident that our model's performance is only marginally better than random chance. This suggests that the model may not effectively discriminate between positive and negative outcomes. To enhance its predictive power and better address the minority class, we may need to further refine our model or consider additional strategies such as resampling techniques or employing different algorithms.

In [ ]:
# Saving final model
from joblib import dump, load
dump(clf, 'final_model.joblib')